In [1]:
# LSTM v1 - Simple Lstm experiments
from sequences.label_dictionary import LabelDictionary
dic = LabelDictionary([], start_index=0)

dic.add("a")
dic.add("b")
print dic

print dic.names

assert("a" == dic.get_label_name(dic["a"]))
assert("b" == dic.get_label_name(dic["b"]))

{'a': 0, 'b': 1}
{0: 'a', 1: 'b'}


In [2]:
%load_ext autoreload
%autoreload 2

import readers.tac_prep_corpus as tc
from readers.tac_prep_corpus import *

from sequences.sequence_list import SequenceList

print data_dir

readers/../data/


In [3]:
data_tac2014_train = data_dir+"/clear_data/data_tac2014_train.json"
data_tac2014_eval = data_dir+"/clear_data/data_tac2014_eval.json"
data_tac2015_train = data_dir+"/clear_data/data_tac2015_train.json"
data_tac2015_eval = data_dir+"/clear_data/data_tac2015_eval.json"

Vocab and statistics

In [4]:
# reload(readers.tac_prep_corpus)
word_counts = TacPrepJsonCorpus.word_counts_from_jsonfiles(json_files = [data_tac2014_train, data_tac2014_eval, data_tac2015_train])

# print "Top 10 tokens:"
print word_counts[:10]

[(u'.', 18754), (u'the', 16829), (u',', 15016), (u'to', 9928), (u'a', 8518), (u'of', 8155), (u'and', 7597), (u'in', 6388), (u'I', 4800), (u'that', 4490)]


In [5]:
# word_counts.sort(key = lambda x:x[1], reverse=True)
print "Vocab size:%s" % len(word_counts)
print "Tokens with freq>=%s : %s" % (2, len( [xx for xx in word_counts if xx[1]>=2]))

Vocab size:20459
Tokens with freq>=2 : 15097


In [6]:
unknown_word = "<UNKNWN>"
pad_word = "<PAD>"
# clear the vocab
vocab = [xx[0] for xx in word_counts]
vocab.insert(0, unknown_word)
vocab.insert(0, pad_word)
print vocab[:10]

vocab_dict = LabelDictionary(vocab, start_index=0)

['<PAD>', '<UNKNWN>', u'.', u'the', u',', u'to', u'a', u'of', u'and', u'in']


In [7]:
print len([xx for xx in vocab if ("/" in xx)or("\\" in xx)])


400


# Load embeddings

In [8]:
from gensim.models import Word2Vec

word2vec_model_filename = data_dir+"../resources/external/w2v_embeddings/" + "qatarliving_qc_size20_win10_mincnt5_rpl_skip1_phrFalse_2016_02_23.word2vec.bin"
word2vec_model_binary = False

# word2vec_model = Word2Vec.load_word2vec_format(word2vec_model_filename, binary=word2vec_model_binary)
word2vec_model = Word2Vec.load(word2vec_model_filename)

index2word = word2vec_model.index2word
index2word_set = set(index2word)
print len(index2word)

104633


In [9]:
print index2word[:10]

[u'distributers', u'mogalina', u'gag', u'woods', u'spiders', u'woody', u'trawling', u'regularize', u'canes', u'erfectly']


In [10]:
# words in training data which are in the w2v embeddings dictionary

print "Tokens in w2v freq>=%s : %s" % (2, len( [xx for xx in word_counts if xx[1]>1 or xx[0] in index2word_set or xx[0].lower() in index2word_set]))

Tokens in w2v freq>=2 : 18906


In [11]:
# merge the index2word with the dictionary of the corpus

word_counts_dict = dict(word_counts)

full_dict_word_counts = {}
for wi in range(0, len(index2word)):
    if index2word[wi] in word_counts_dict:
        full_dict_word_counts[index2word[wi]] = word_counts_dict[index2word[wi]]
    else:
        full_dict_word_counts[index2word[wi]] = 0
        
full_dict_word_counts = [(k,v) for (k,v) in full_dict_word_counts.items()]
full_dict_word_counts.sort(key=lambda f: f[1], reverse=True)

print full_dict_word_counts[:10]
    

[(u'the', 16829), (u'to', 9928), (u'a', 8518), (u'of', 8155), (u'and', 7597), (u'in', 6388), (u'that', 4490), (u'is', 4143), (u'for', 3738), (u'was', 3434)]


In [12]:
# from Word2Vec_AverageVectorsUtilities import AverageVectorUtilities
import numpy as np
import time as ti
def get_average_vector(words, model, num_features, index2word_set):
    #function to average all words vectors in a given paragraph
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0

    #list containing names of words in the vocabulary
    #index2word_set = set(word2vec_model.index2word) this is moved as input param for performance reasons
    for word in words:
        if word in index2word_set:
            nwords = nwords+1
            featureVec = np.add(featureVec, model[word])

    if(nwords>0):
        featureVec = np.divide(featureVec, nwords)
    return featureVec

index2word_set=set(index2word)
st = ti.time()
unknown_vec = get_average_vector(index2word, word2vec_model, word2vec_model.syn0.shape[1], index2word_set)
print ti.time()-st

print "unknown_vec:%s" % unknown_vec

0.76778793335
unknown_vec:[ 0.07160316  0.06742626  0.15077273 -0.07981906 -0.11572608 -0.09256729
 -0.08221466 -0.09356559  0.01780725 -0.0515666   0.05499313 -0.10132896
 -0.09117364  0.16332768 -0.0177103  -0.07968703  0.07441449  0.10293729
 -0.13497648 -0.10935152]


In [13]:
from VocabEmbedding_Utilities import VocabEmbeddingUtilities

st = ti.time()

vocab_and_embeddings = VocabEmbeddingUtilities.get_embeddings_for_vocab_from_model(vocabulary=vocab_dict, 
                                                            embeddings_type='w2v', 
                                                            embeddings_model=word2vec_model, 
                                                            embeddings_size=word2vec_model.syn0.shape[1])

#set the unknown embeddings to the average of all word embeddings
vocab_and_embeddings["embeddings"][vocab_and_embeddings["vocabulary"][unknown_word],:] = unknown_vec

# print unknown_vec
# print vocab_and_embeddings["embeddings"][vocab_and_embeddings["vocabulary"][unknown_word]]
print "Done in %s s" %(ti.time()-st)
# print vocab_and_embeddings["vocabulary"]
# print vocab_and_embeddings["embeddings"]

Building embeddings...
20461
Done in 0.323060035706 s


# Load the corpus

In [14]:
max_sent_len = 1000 
max_nr_sent = 100000
max_sent_len=max_sent_len
max_nr_sent=max_nr_sent
update_vocab=False
update_tags=False

unknown_tag = u'O'
mapping_file = None
data_x_fieldname = "tokens"
data_y_fieldname = "labels_event"



In [15]:
st = ti.time()
labels_counts = TacPrepJsonCorpus.word_counts_from_jsonfiles(json_files = [data_tac2014_train, data_tac2014_eval, data_tac2015_train], data_fieldname=data_y_fieldname)
print "Done in %s s.." %(ti.time()-st)
print labels_counts

labels_sorted_alpha = sorted(labels_counts, key=lambda xx:xx[0])
labels_lst = [unknown_tag] + [xx[0] for xx in labels_sorted_alpha if xx[0] <> unknown_tag]



Done in 17.8586909771 s..
[(u'O', 402099), (u'B-EVENT', 15656), (u'I-EVENT', 553)]


In [16]:
print labels_lst

[u'O', u'B-EVENT', u'I-EVENT']


In [17]:
def count_non_zero_label(test_seq, zero_label=0):
    cnt = 0
    for item in test_seq:
        for lbl in item.y:
            if lbl != zero_label:
                cnt += 1
                # print item.y
                break

    return cnt

In [18]:
# Use indexes that start from 1 if you use TF dynamic sequences!
tag_start_index = 1
vocab_start_index = 0

In [19]:
corpus_vocab_input = LabelDictionary()
corpus_vocab_input.set_dict(vocab_and_embeddings["vocabulary"])

tac_corpus = TacPrepJsonCorpus([], labels_lst, 
                               tag_start_index=tag_start_index, 
                               vocab_start_index=vocab_start_index)

tac_corpus.set_word_dict(corpus_vocab_input)

print tac_corpus.word_dict[unknown_word]
print tac_corpus.sequence_list.x_dict[unknown_word]

1
1


In [20]:
test_seq, test_seq_meta = tac_corpus.read_sequence_list_tac_json([data_tac2015_eval], max_sent_len=max_sent_len, max_nr_sent=max_nr_sent, 
                         update_vocab=update_vocab, update_tags=update_tags, unknown_word=unknown_word, 
                         unknown_tag=unknown_tag, mapping_file=mapping_file, 
                         data_x_fieldname=data_x_fieldname,
                         data_y_fieldname=data_y_fieldname)

readers/../data//clear_data/data_tac2015_eval.json


In [21]:
# print tac_corpus.word_dict

    

In [22]:

print "All sents:%s" % len(test_seq)
print "With non zero labels:%s" % count_non_zero_label(test_seq, zero_label = tag_start_index)
print test_seq_meta[0]["tokens"]
print test_seq[0].x
print test_seq[0].y

print "data fields:"
for k,v in  test_seq_meta[0].iteritems():
    print k
# print [tac_corpus.word_dict[a] for a in test_seq[0].x]


All sents:4164
With non zero labels:2646
[u'regime', u'<img\xa0src="http://www.amnesty.org/sites/impact.amnesty.org/files/imagecache/news-highlight/brazil-amnesty-law%2025.04.12.JPG"/>', u'The', u'efforts', u'of', u'federal', u'prosecutors', u'to', u'initiate', u'criminal', u'investigations', u'into', u'past', u'human', u'rights', u'violations', u'marks', u'a', u'crucial', u'moment', u'in', u'Brazil', u"'s", u'history', u',', u'said', u'Amnesty', u'International', u'after', u'federal', u'prosecutors', u'in', u'S\xe3o', u'Paulo', u'charged', u',', u'on', u'24', u'March', u'2012', u',', u'retired', u'Colonel', u'Carlos', u'Alberto', u'Brilhante', u'Ustra', u'and', u'police', u'chief', u'-LRB-', u'delegado', u'-RRB-', u'Dirceu', u'Garvina', u',', u'with', u'the', u'kidnapping', u'of', u'union', u'leader', u'Alu\xedzio', u'Palhano', u'Pedreira', u'Ferreira', u'in', u'1971', u'.']
[1647, 1, 27, 1228, 7, 392, 449, 5, 1, 430, 3638, 121, 406, 563, 260, 2657, 1, 6, 3300, 1927, 9, 776, 15, 584, 

In [23]:
train_seq, train_seq_meta = tac_corpus.read_sequence_list_tac_json([data_tac2014_train, data_tac2015_train], max_sent_len=max_sent_len, max_nr_sent=max_nr_sent, 
                         update_vocab=update_vocab, update_tags=update_tags, unknown_word=unknown_word, 
                         unknown_tag=unknown_tag, mapping_file=mapping_file, 
                         data_x_fieldname=data_x_fieldname,
                         data_y_fieldname=data_y_fieldname)

readers/../data//clear_data/data_tac2014_train.json
readers/../data//clear_data/data_tac2015_train.json


In [24]:
print "All sents:%s" % len(train_seq)
print "With non zero labels:%s" % count_non_zero_label(train_seq, zero_label = tag_start_index)


All sents:12573
With non zero labels:5367


In [25]:
dev_seq, dev_seq_meta = tac_corpus.read_sequence_list_tac_json([data_tac2014_eval], max_sent_len=max_sent_len, max_nr_sent=max_nr_sent, 
                         update_vocab=update_vocab, update_tags=update_tags, unknown_word=unknown_word, 
                         unknown_tag=unknown_tag, mapping_file=mapping_file, 
                         data_x_fieldname=data_x_fieldname,
                         data_y_fieldname=data_y_fieldname)

readers/../data//clear_data/data_tac2014_eval.json


In [26]:
print "All sents:%s" % len(dev_seq)
print "With non zero labels:%s" % count_non_zero_label(dev_seq, zero_label = tag_start_index)

All sents:9041
With non zero labels:3749


In [27]:
print train_seq.y_dict

# Redo indices
# train_seq, test_seq, dev_seq = tc.compacify(train_seq, test_seq, dev_seq,
#                                             default_vocab_dict=[unknown_word], 
#                                             vocab_start_index=vocab_start_index, 
#                                             default_tag_dict=[unknown_tag], 
#                                             tag_start_index=tag_start_index,
#                                             theano=True)



# Get number of words and tags in the corpus
nr_words = len(train_seq.x_dict)
nr_tags = len(train_seq.y_dict)


{u'B-EVENT': 2, u'O': 1, u'I-EVENT': 3}


In [28]:
for k,v in train_seq.x_dict.iteritems():
    #print k,v
    if v>nr_words:
        print (k,v)
        
print nr_words
print nr_tags

20461
3


# Run the lstm

In [29]:
# import lstm_v1 as lstm_v1
# import theano
# import theano.tensor as T

In [30]:
emb_size=50
hidden_size=50
SEED = 42

lrate = 0.0001

nr_iterations = 5
batch_size = 50
print "nr_words:%s" % nr_words
print "emb_size:%s" % emb_size
print "hidden_size:%s" % hidden_size
print "nr_tags:%s" % nr_tags
print "lrate:%s" % lrate



nr_words:20461
emb_size:50
hidden_size:50
nr_tags:3
lrate:0.0001


# Tensorflow 

In [31]:
import numpy as np
import tensorflow as tf



## Tensorflow padding and batching

In [32]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

# x = tf.placeholder(dtype=tf.int64, shape=[None, None], name = "x")
# y = tf.placeholder(dtype=tf.int64, shape=[None, None], name = "y")
# seq_len = tf.placeholder(dtype=tf.int64, shape=[None], name = "seq_len")

# batched_data = tf.train.batch(
#     tensors=[x, y, seq_len],
#     batch_size = batch_size,
#     dynamic_pad = True,
#     name="batch_data"
# )

In [33]:
# # data_x = np.array([np.array(a.x) for a in train_seq])
# # data_seq_len = np.array([len(a.x) for a in train_seq])
# # data_y = np.array([np.array(a.y) for a in train_seq])

# data_x = tf.convert_to_tensor([a.x.tolist() for a in train_seq], dtype=tf.int64)
# data_seq_len = tf.convert_to_tensor([len(a.x) for a in train_seq], dtype=tf.int64)
# data_y = tf.convert_to_tensor([a.y.tolist() for a in train_seq], dtype=tf.int64)

# print data_x[:5]
# print data_seq_len[:5]
# print data_y[:5]

In [34]:
# print(batched_data)
# print type(data_x)
# print type(data_y)
# print type(data_seq_len)

# #tf.reset_default_graph()

# # Run the graph
# # tf.contrib.learn takes care of starting the queues for us
# # res = sess.run(batched_data[0], batched_data[1], batched_data[2] , feed_dict={x:data_x, y:data_y, seq_len:data_seq_len})
# result = tf.contrib.learn.run_n(
#     {
#         "batched_data_x": batched_data[0],
#         "batched_data_y": batched_data[1],
#         "batched_data_seq_len": batched_data[2]
#     },
#     n=1,
#     feed_dict={x:data_x, 
#                seq_len:data_seq_len,
#                y:data_y})


# Print the result
# print("Batch shape: {}".format(res[0]["y"].shape))
# print(res[0]["y"])

## Bi LSTM
    

In [35]:
embeddings_number = nr_words
embedding_size = emb_size
embedding_size = 20 # current w2v embeddings
n_classes = nr_tags
learning_rate=0.1

print "embeddings_number:%s" % embeddings_number
print "embedding_size:%s" % embedding_size
print "n_classes:%s" % n_classes
print "learning_rate:%s" % learning_rate

tf.reset_default_graph()

# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of forward + backward cells
    #'out': tf.Variable(tf.random_uniform([2*hidden_size, n_classes], minval=-1, maxval=1, dtype=tf.float64), name="out_w", dtype=tf.float64)
    'out': tf.Variable(tf.truncated_normal([2*hidden_size, n_classes], stddev=0.1, dtype=tf.float64), trainable=True, name="out_w", dtype=tf.float64)
}
biases = {
    # 'out': tf.Variable(tf.random_uniform([n_classes], minval=-1, maxval=1, dtype=tf.float64), name="out_b", dtype=tf.float64)
    'out': tf.Variable(tf.truncated_normal([n_classes], stddev=0.1, dtype=tf.float64), trainable=True, name="out_b", dtype=tf.float64)
}

input_x = tf.placeholder(dtype=tf.int64, shape=[None, None], name = "input_x")
input_y = tf.placeholder(dtype=tf.int64, shape=[None, None], name = "input_y") # this is not onehot!
input_seq_len = tf.placeholder(dtype=tf.int64, shape=[None], name = "input_seq_len")

with tf.device('/cpu:0'), tf.name_scope("embeddings"):
    embeddings_placeholder = tf.placeholder(tf.float64, shape=[embeddings_number, embedding_size])
    # embeddings_tuned = tf.Variable(embeddings_placeholder)
    
    # embeddings random, tuned
    # embeddings_tuned =tf.Variable(tf.truncated_normal([embeddings_number, embedding_size], stddev=0.1, dtype=tf.float64), trainable=False, name="embeddings", dtype=tf.float64)
    
    # embeddings, loaded, tuned
    embeddings_tuned =tf.Variable(vocab_and_embeddings["embeddings"], trainable=True, name="embeddings_input", dtype=tf.float64)
    embedded_chars = tf.nn.embedding_lookup(embeddings_tuned, input_x)
    # embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)


def BiLSTM_dynamic(x_embedd, input_seq_len, shared_fw_bw=False,
                   use_peepholes=False, cell_clip=None,
                   # initializer=None, 
                   num_proj=None, proj_clip=None,
                   num_unit_shards=1, num_proj_shards=1,
                   forget_bias=1.0, state_is_tuple=True,
                   activation=tf.tanh):
    with tf.name_scope("BiLSTM"):
        with tf.variable_scope('forward'):
            cell_fw = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True,
                                               use_peepholes=use_peepholes, cell_clip=cell_clip,
                                               initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=42),
                                               num_proj=num_proj, proj_clip=proj_clip,
                                               num_unit_shards=num_unit_shards, num_proj_shards=num_proj_shards,
                                               forget_bias=forget_bias,
                                               activation=activation)

        with tf.variable_scope('backward'):    
            cell_bw = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True,
                                               use_peepholes=use_peepholes, cell_clip=cell_clip,
                                               initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=24), 
                                               num_proj=num_proj, proj_clip=proj_clip,
                                               num_unit_shards=num_unit_shards, num_proj_shards=num_proj_shards,
                                               forget_bias=forget_bias,
                                               activation=activation)

        outputs, states  = tf.nn.bidirectional_dynamic_rnn(
            cell_fw=cell_fw,
            cell_bw=cell_fw if shared_fw_bw else cell_bw,
            dtype=tf.float64,
            sequence_length=input_seq_len,
            inputs=x_embedd)

    return outputs, states

# Dynamic BiLSTM outputs and states
outputs, states = BiLSTM_dynamic(x_embedd=embedded_chars,
                                 input_seq_len=input_seq_len,
                                 cell_clip=None,
                                 shared_fw_bw=True)
output_fw, output_bw = outputs
states_fw, states_bw = states


# we use concatenation over the fw and bw outputs - some approaches use sum?
# output_concat = tf.concat(2, outputs) - we have to reverse and concat
output_bw_reversed = tf.reverse(output_bw,dims=(False, False, True))
outputs_rev = [output_fw, output_bw_reversed]

output_concat = tf.concat(2, outputs_rev)

# Calculate logits and probs
# Reshape so we can calculate them all at once
bi_output_concat_flat = tf.reshape(output_concat, [-1, 2 * hidden_size])
# bi_output_concat_flat_clipped = tf.clip_by_value(bi_output_concat_flat, -1., 1.)

logits_flat = tf.batch_matmul(bi_output_concat_flat, weights["out"]) + biases["out"]
# logits_flat = tf.clip_by_value(logits_flat, -1., 1.)
probs_flat = tf.nn.softmax(logits_flat)

print "outputs[-1].shape:%s" % outputs[-1]#
print "weights[\"out\"]].shape:%s" % weights["out"]#.shape
print "biases[\"out\"].shape:%s" % biases["out"]#.shape

# print pred

#make y flat so it match pred shape
input_y_flat =  tf.reshape(input_y, [-1])
print "input_y_flat:%s" % input_y_flat
# # Define loss and optimizer

def tf_nan_to_zeros_float64(tensor):
    return tf.select(tf.is_nan(tensor), tf.zeros(tf.shape(tensor), dtype=tf.float64),tensor)
# sparse softmax cros entropy - aka - calculates on non-onehot y!
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits_flat, input_y_flat) 
# replace nans with zeros
losses = tf_nan_to_zeros_float64(losses)

learn_rate = tf.Variable(learning_rate, trainable=False)

#Applying compute and apply gradients
optimizer = tf.train.AdamOptimizer(learning_rate=learn_rate)

gvs = optimizer.compute_gradients(losses)
print "gradients:"
for grad, var in gvs:
    print "%s - %s" % (grad.name, var)
    
capped_gvs = [(tf.clip_by_value(tf_nan_to_zeros_float64(grad), -1., 1.), var) for grad, var in gvs]

apply_grads_op=optimizer.apply_gradients(capped_gvs)


## Calculate the accuracy
## Used during training
# Mask the losses - padded values are zeros
mask = tf.sign(tf.cast(input_y_flat, dtype=tf.float64))
print "mask:%s" % mask
masked_losses = mask * losses

# Bring back to [batch, class_num] shape
masked_losses = tf.reshape(masked_losses,  tf.shape(input_y))

input_seq_len_float = tf.cast(input_seq_len, dtype=tf.float64)

# Calculate mean loss - depending on the dynamic number of elements
mean_loss_by_example = tf.reduce_sum(masked_losses, reduction_indices=1) / input_seq_len_float
mean_loss = tf.reduce_mean(mean_loss_by_example)

# # Evaluate model
print "probs_flat:%s" % probs_flat
print "input_y:%s" % input_y

preds_flat = tf.argmax(probs_flat,1)

preds_non_paddings = tf.gather(preds_flat, tf.where(tf.greater(input_y_flat, [0])))
input_y_non_paddings = tf.gather(input_y_flat, tf.where(tf.greater(input_y_flat, [0])))

preds_y = tf.reshape(preds_flat, tf.shape(input_y))

correct_pred = tf.equal(preds_non_paddings, input_y_non_paddings)

accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float64))

print "The model might be okay :D"



embeddings_number:20461
embedding_size:20
n_classes:3
learning_rate:0.1
outputs[-1].shape:Tensor("BiLSTM/ReverseSequence:0", shape=(?, ?, 50), dtype=float64)
weights["out"]].shape:<tensorflow.python.ops.variables.Variable object at 0x7f9c6d47a210>
biases["out"].shape:<tensorflow.python.ops.variables.Variable object at 0x7f9c6d46bc90>
input_y_flat:Tensor("Reshape_1:0", shape=(?,), dtype=int64)
gradients:
gradients/BatchMatMul_grad/tuple/control_dependency_1:0 - <tensorflow.python.ops.variables.Variable object at 0x7f9c6d47a210>
gradients/add_grad/tuple/control_dependency_1:0 - <tensorflow.python.ops.variables.Variable object at 0x7f9c6d46bc90>
gradients/embeddings/embedding_lookup_grad/Reshape:0 - <tensorflow.python.ops.variables.Variable object at 0x7f9c6b444250>
gradients/BiLSTM/BiRNN_FW/BiRNN_FW/while/LSTMCell/MatMul/Enter_grad/b_acc_3:0 - <tensorflow.python.ops.variables.Variable object at 0x7f9c6b3b7fd0>
gradients/BiLSTM/BiRNN_FW/BiRNN_FW/while/LSTMCell/BiasAdd/Enter_grad/b_acc_3:0

In [36]:
# embeddings
# embeddings_random = 0.01*np.random.uniform(size=(embeddings_number, embedding_size))
embeddings_random = vocab_and_embeddings["embeddings"]
# embeddings_random[0][0] = float('nan')

nan_vals = 0

print np.isnan(np.sum(embeddings_random))
# for i in range(len(embeddings_random)):
#     for j in range(embeddings_random.shape[1]):
#         if np.isNan(embeddings_random[i][j]):
#             nan_vals +=1
# print nan_vals

False


In [37]:
#Padding
def pad(seq, pad_value, to_size):
    pad_seq = []
    if len(seq) > to_size:
        pad_seq = seq[:to_size]
    else:
        pad_seq = seq[:] + [pad_value]*(to_size - len(seq))
        
    return pad_seq

print pad([1,2,3], 0, 8)
print pad([1,2,3,4,5,6,7,8], 0, 8)
print pad([1,2,3,4,5,6,7,8,9,10], 0, 8)
        


[1, 2, 3, 0, 0, 0, 0, 0]
[1, 2, 3, 4, 5, 6, 7, 8]
[1, 2, 3, 4, 5, 6, 7, 8]


In [38]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix([0, 0, 1, 1], [0,0,1,0])
classes_dict = dict([(v,k) for (k,v) in train_seq.y_dict.items()])
print classes_dict

def print_acc_from_conf_matrix(conf_matrix, classes_dict):
    for lblid in range(conf_matrix.shape[0]):
        overl_cnt = np.sum(conf_matrix[lblid])
        true_cnt = conf_matrix[lblid][lblid]
        lbl_acc = float(true_cnt)/float(overl_cnt) if overl_cnt > 0 else 1
        lbl_name = classes_dict[lblid] if lblid in classes_dict else "None"
        print "%s (%s) accuracy = %s"%(lbl_name.ljust(10), lblid, lbl_acc)

print_acc_from_conf_matrix(conf_matrix, classes_dict)
from sklearn.metrics import precision_recall_fscore_support

{1: u'O', 2: u'B-EVENT', 3: u'I-EVENT'}
None       (0) accuracy = 1.0
O          (1) accuracy = 0.5


In [39]:
import time as ti
from sklearn.metrics import precision_recall_fscore_support

# Initializing the variables
init = tf.initialize_all_variables()
sess = tf.InteractiveSession()

sess.run(init, feed_dict = None)

def prepare_batch_data(data, pad_value=0):
    batch_data_seqlens = [len(a.x) for a in data]
    max_len = max(batch_data_seqlens)

#     batch_data_padded_x = [pad(a.x.tolist(), pad_value, max_len) for a in data]
#     batch_data_padded_y = [pad(a.y.tolist(), pad_value, max_len) for a in data]
    
    batch_data_padded_x = [pad(a.x, pad_value, max_len) for a in data]
    batch_data_padded_y = [pad(a.y, pad_value, max_len) for a in data]
    # batch_data_padded_x_embedd = [[pad(a.x.tolist(), pad_value, max_len)] for a in batch_data]
    # batch_data_padded_y_mask = [[1 if item>0 else 0 for item in pad(a.y.tolist(), pad_value, max_len)] for a in batch_data]

    return batch_data_padded_x, batch_data_padded_y, batch_data_seqlens

def train_step(x_batch, y_batch, x_batch_seq_len):

    feed_dict = {
        embeddings_placeholder : embeddings_random,
        input_x : x_batch, # batch_data_padded_x,
        input_y : y_batch, # batch_data_padded_y,
        input_seq_len: x_batch_seq_len # batch_data_seqlens
    }

    #_, \
    _,\
    res_cost, \
    res_acc, \
    res_output_y,\
    res_mean_loss_by_example, \
    res_logits_flat,\
    res_losses, \
    res_probs_flat, \
    res_input_seq_len_float, \
    res_input_y_flat, res_bi_output_concat_flat, \
    res_output_fw, res_output_bw_reversed, res_embedded_chars, \
    res_weights_out, res_biases_out, \
    res_capped_gvs, \
    res_gvs \
    = sess.run([
                apply_grads_op,
                mean_loss, 
                accuracy, 
                preds_y, 
                mean_loss_by_example, 
                logits_flat,
                losses, 
                probs_flat,
                input_seq_len_float,
                input_y_flat, bi_output_concat_flat,
                output_fw, output_bw_reversed, embedded_chars,
                weights["out"], biases["out"],
                capped_gvs,
                gvs], 
               feed_dict=feed_dict)

    check_for_nans = True

    
    non_zero_input = [oy for oy in y_batch if 2 in oy]
    # print "Non_zero_input:%s" % len(non_zero_input)
    non_zero_preds = [oy for oy in res_output_y if 2 in oy]
    # print "Non-zero preds:%s" % len(non_zero_preds)
    
    # print non_zero_preds[:3]
    if check_for_nans:
        # print "res_input_seq_len_float:%s" % res_input_seq_len_float
        has_nans = False
        # res_output_concat =  sess.run(output_concat, feed_dict=feed_dict)
        # print res_output_concat.shape
        if np.isnan(np.sum(res_embedded_chars)):
            has_nans = True
            print "[res_embedded_chars] has nans!"

        if np.isnan(np.sum(res_output_fw)):
            has_nans = True
            print "[res_output_fw] has nans!"

        if np.isnan(np.sum(res_output_bw_reversed)):
            has_nans = True
            print "[res_output_bw_reversed] has nans!"

        if np.isnan(np.sum(res_bi_output_concat_flat)):
            has_nans = True
            print "[res_bi_output_concat_flat] has nans!"

        if np.isnan(np.sum(res_weights_out)):
            has_nans = True
            print "[res_weights_out] has nans!"

        if np.isnan(np.sum(res_biases_out)):
            has_nans = True
            print "[res_biases_out] has nans!"            

        if np.isnan(np.sum(res_logits_flat)):
            has_nans = True
            print "[res_logits_flat] has nans!"

        if np.isnan(np.sum(res_probs_flat)):
            has_nans = True
            print "[res_probs_flat] has nans!"

        if np.isnan(np.sum(res_losses)):
            has_nans = True
            print "[res_losses] has nans!"

        if np.isnan(np.sum(res_mean_loss_by_example)):
            has_nans = True
            print "[res_mean_loss_by_example] has nans!"

        if np.isnan(res_cost):
            has_nans = True
            print "[res_cost] has nans!"

#         if np.isnan(res_gvs):
#             has_nans = True
#             print "[res_gvs] has nans!"

#         if np.isnan(res_capped_gvs):
#             has_nans = True
#             print "[res_capped_gvs] has nans!"

        if has_nans:
            print "res_weights_out:%s" % res_weights_out[:5]
            print "res_biases_out:%s" % res_biases_out
            print "batch_data_padded_x:%s" % batch_data_padded_x[0]
            print "res_embedded_chars:%s" % len(res_embedded_chars), len(res_embedded_chars[0])
            print "res_output_fw:%s" % res_output_fw[0]
            print "res_output_bw_reversed:%s" % res_output_bw_reversed[0]
            # print "res_bi_output_concat_flat:%s" % res_bi_output_concat_flat
            print "res_input_y_flat:%s" % res_input_y_flat
            print "res_logits_flat:%s" % res_logits_flat
            print "res_res_probs_flat:%s" % res_probs_flat
            print "res_mean_loss_by_example:%s" % res_mean_loss_by_example
            print "res_losses:%s" % res_losses
            print "res_capped_gvs:%s" % res_capped_gvs
            print "res_gvs:%s" % res_gvs

            # print losses by example
            res_losses_reshaped = res_losses.reshape([50, res_losses.shape[0]/50])
            for i in range(len(res_losses_reshaped)):
                if np.isnan(np.sum(res_losses_reshaped[i])):
                    print res_losses_reshaped[i]
            raise Exception("Fucking NANS!")

    return res_cost, res_acc
        
def dev_step(x_batch, y_batch, x_batch_seq_len):
    
    feed_dict = {
        embeddings_placeholder : embeddings_random,
        input_x : x_batch, # batch_data_padded_x,
        input_y : y_batch, # batch_data_padded_y,
        input_seq_len: x_batch_seq_len # batch_data_seqlens
    }

    # _ ,\
    res_cost, \
    res_acc, \
    res_output_y,\
    res_mean_loss_by_example, \
    res_logits_flat,\
    res_losses, \
    res_probs_flat, \
    res_input_seq_len_float, \
    res_input_y_flat, res_bi_output_concat_flat, \
    res_output_fw, res_output_bw_reversed, res_embedded_chars, \
    res_weights_out, res_biases_out, \
    res_capped_gvs, \
    res_gvs \
    = sess.run([
                # apply_grads_op, - commented so does not apply gradients
                mean_loss, 
                accuracy, 
                preds_y,
                mean_loss_by_example, 
                logits_flat,
                losses, 
                probs_flat,
                input_seq_len_float,
                input_y_flat, bi_output_concat_flat,
                output_fw, output_bw_reversed, embedded_chars,
                weights["out"], biases["out"],
                capped_gvs,
                gvs
                ], 
               feed_dict=feed_dict)

    # print " Dev cost %2.2f | Dev batch acc %2.2f %% in %s\n" % (res_cost, res_acc, ti.time()-start),

    return res_output_y, res_cost, res_acc

pad_value = 0
train_batches_cnt = len(train_seq)/batch_size
dev_batches_cnt = len(dev_seq)/batch_size
test_batches_cnt = len(test_seq)/batch_size

print "Batches:%s" % train_batches_cnt
#batches_cnt = min(batches_cnt, 1)
epochs_cnt = 1
#train with batches
max_len = 0
print "Start training in %s epochs" % epochs_cnt
for epoch in range(0, epochs_cnt):
    print "Epoch %d:" % (epoch+1),
    start_epoch = ti.time()
    
    for i in range(0, train_batches_cnt):
        # print"Batch %s =================" % (i+1)
        
        #Get the batch
        batch_data = train_seq[i*batch_size:i*batch_size+batch_size]
        batch_data_padded_x, batch_data_padded_y, batch_data_seqlens = prepare_batch_data(data=batch_data)

        #Do the train step
        start = ti.time()
        cost, acc = train_step(x_batch = batch_data_padded_x, y_batch = batch_data_padded_y, x_batch_seq_len = batch_data_seqlens)
        # print " Train cost %2.2f | Train batch acc %2.2f %% in %s\n" % (cost, acc, ti.time()-start)
        
    print " train epoch time %s " % (ti.time()-start_epoch)
    
    #Dev eval - once per epoch
    print "Dev set:"
    input_y_all = []
    pred_y_all = []
    
    
    for i in range(0, dev_batches_cnt):
        batch_data = dev_seq[i*batch_size:i*batch_size+batch_size]
        batch_data_padded_x, batch_data_padded_y, batch_data_seqlens = prepare_batch_data(data=batch_data)
    
        res_pred_y, cost, acc = dev_step(x_batch = batch_data_padded_x, y_batch = batch_data_padded_y, x_batch_seq_len = batch_data_seqlens)
        
#         print batch_data_padded_y[10]
#         print res_pred_y[10]
        
        for j in range(0, batch_size):
            input_y_all.extend(batch_data_padded_y[j][:batch_data_seqlens[j]])
            pred_y_all.extend(res_pred_y[j][:batch_data_seqlens[j]])
            
            
    # eval
    print "Confusion matrix:"
    conf_matrix = confusion_matrix(input_y_all, pred_y_all)
    print conf_matrix
    print "Accuracy by class:"
    print_acc_from_conf_matrix(conf_matrix, classes_dict)
    
#     print "Class  : P, R, F-Score:" 
#     print "B-Event:"
#     print precision_recall_fscore_support(input_y_all, pred_y_all, average=None, pos_label=2)
#     print "I-Event:"
#     print precision_recall_fscore_support(input_y_all, pred_y_all, average=None, pos_label=3)
#     
    print " Dev cost %2.2f | Dev acc %2.2f %% in %s\n" % (cost, acc, ti.time()-start)
    
    #################
    ### Test eval ###
    #################
    print "Test set:"
    input_y_all = []
    pred_y_all = []
    pred_y_all_varlength = []
    
    for i in range(0,test_batches_cnt):
        batch_data = test_seq[i*batch_size:i*batch_size+batch_size]
        batch_data_padded_x, batch_data_padded_y, batch_data_seqlens = prepare_batch_data(data=batch_data)
    
        res_pred_y, cost, acc = dev_step(x_batch = batch_data_padded_x, y_batch = batch_data_padded_y, x_batch_seq_len = batch_data_seqlens)
        
#         print batch_data_padded_y[10]
#         print res_pred_y[10]
        
        for j in range(0, batch_size):
            input_y_all.extend(batch_data_padded_y[j][:batch_data_seqlens[j]])
            pred_y_all.extend(res_pred_y[j][:batch_data_seqlens[j]])
            
            #output to be formatted
            pred_y_all_varlength.append(res_pred_y[j][:batch_data_seqlens[j]])
            
            
    # eval
    print "Confusion matrix:"
    conf_matrix = confusion_matrix(input_y_all, pred_y_all)
    print conf_matrix
    print "Accuracy by class:"
    print_acc_from_conf_matrix(conf_matrix, classes_dict)
    
#     print "Class  : P, R, F-Score:" 
#     print "B-Event:"
#     print precision_recall_fscore_support(input_y_all, pred_y_all, average=None, pos_label=2)
#     print "I-Event:"
#     print precision_recall_fscore_support(input_y_all, pred_y_all, average=None, pos_label=3)
#     
    print " Test cost %2.2f | Dev acc %2.2f %% in %s\n" % (cost, acc, ti.time()-start)
    

    

Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f9c96817710>> ignored


Batches:251
Start training in 1 epochs
Epoch 1:  train epoch time 157.982012033 
Dev set:
Confusion matrix:
[[     0      0      0      0]
 [     2 164676    865      0]
 [     2   2936   3351      0]
 [     0    269     77      0]]
Accuracy by class:
None       (0) accuracy = 1
O          (1) accuracy = 0.994762690056
B-EVENT    (2) accuracy = 0.53283510892
I-EVENT    (3) accuracy = 0.0
 Dev cost 0.08 | Dev acc 0.98 % in 196.893202066

Test set:
Confusion matrix:
[[    0     0     0]
 [    2 91493   291]
 [    1  3648  1979]]
Accuracy by class:
None       (0) accuracy = 1
O          (1) accuracy = 0.996807792038
B-EVENT    (2) accuracy = 0.351634683724
 Test cost 0.16 | Dev acc 0.96 % in 308.662307024



# Evaluate and export

In [51]:
# Extract event nuggets from labeled sentences
def extract_eventnuggets(data_meta, use_per_doc_event_indexing=True):
    docs_with_events = []
    
    # metadata fields:
    # tokens
    # deps_basic
    # lemmas
    # deps_cc
    # pos
    # parse
    # file_name
    # file_id
    # char_offsets
    # labels_realis
    # labels_type_full
    # labels_event

    curr_doc_id = ""
    event_id = 0
    
    curr_doc_data = {}
    for data_item in data_meta:
        doc_id = data_item["file_id"]
        if doc_id <> curr_doc_id:
            if curr_doc_id<>"":
                docs_with_events.append(curr_doc_data)
                
            # start new doc data export
            curr_doc_id = doc_id
            
            curr_doc_data = {}
            curr_doc_data["doc_id"] = curr_doc_id
            curr_doc_data["file_id"] = curr_doc_id
            curr_doc_data["file_name"] = data_item["file_name"]
            
            curr_doc_data["event_nuggets"] = []
            
        lbl_event_bevent = u"B-EVENT"
        lbl_event_ievent = u"I-EVENT"

        lbl_bevent_cnt = sum([1 for xx in data_item["labels_event"] if xx == lbl_event_bevent])
        # print "lbl_bevent_cnt:%s" % lbl_bevent_cnt
        lbl_ievent_cnt = sum([1 for xx in data_item["labels_event"] if xx == lbl_event_ievent])
        # print "lbl_ievent_cnt:%s" % lbl_ievent_cnt

        curr_event_token_i = 0
        sent_len = len(data_item["tokens"])
        for i in range(0, sent_len):
            if data_item["labels_event"][i] == lbl_event_bevent:
                
                event_id += 1
                curr_event_token_i = i 
                curr_event_nugget = {}
                curr_event_nugget["event_id"]="E%s"%event_id

                #max search for I-EVENT for longer span events
                max_span_tokens_search = 5 # In the data there is no span longer the 4 - we set 5 for convenience. 
                                           # Howerver longer would span would be an error.
                last_ievent_idx = 0
                if lbl_ievent_cnt>0:
                    for j in range(i+1, min(i + max_span_tokens_search, sent_len-1)):
                        if data_item["labels_event"][j] == lbl_event_ievent:
                            last_ievent_idx = j
                            lbl_ievent_cnt -= 1

                        elif data_item["labels_event"] == lbl_event_bevent:
                            break

                if last_ievent_idx == 0:
                    last_ievent_idx = curr_event_token_i

                nugget_span = (100000,0)
                nugget_txt = ""

                curr_event_nugget["tokens"] = []
                curr_event_nugget["char_offsets"] = []
                range_curr = range(curr_event_token_i, last_ievent_idx+1)
                
                # print range_curr
                for k in range_curr:
                    token = data_item["tokens"][k]
                    curr_event_nugget["tokens"].append(token)

                    char_offset = data_item["char_offsets"][k]
                    curr_event_nugget["char_offsets"].append(char_offset)

                    prev_token_span_to = nugget_span[1]
                    # add spaces if tokens are not connected to each other "I"<s>"eat"<s>"ham"
                    if prev_token_span_to > 0 and  (char_offset[1] - prev_token_span_to - 1) > 0:
                        nugget_txt += " " * (char_offset[0] - prev_token_span_to)

                    nugget_txt += token

                    nugget_span = (min(nugget_span[0], char_offset[0]), char_offset[1])

                curr_event_nugget["text"] = nugget_txt
                curr_event_nugget["span"] = nugget_span
                curr_event_nugget["realis"] = data_item["labels_realis"][i][2:] # Assign the value of B-EVENT
                curr_event_nugget["type_full"] = data_item["labels_type_full"][i][2:] # Assign the value of B-EVENT
                
                #Confidence score
                curr_event_nugget["span_confidence"] = 1.0
                curr_event_nugget["realis_confidence"] = 1.0
                curr_event_nugget["type_confidence"] = 1.0
                
                # print "E%s - %s" %(event_id,curr_event_nugget)
                curr_doc_data["event_nuggets"].append(curr_event_nugget)
        
    # add the last document
    docs_with_events.append(curr_doc_data)
     
                        
                        

    return docs_with_events

In [52]:
data_with_meta = [xx for xx in dev_seq_meta if u"I-EVENT" in xx["labels_event"]][:5] # test events with I
docs_with_nuggets = extract_eventnuggets(data_meta=data_with_meta, use_per_doc_event_indexing=True)

# print data_with_meta 
print docs_with_nuggets



[{'file_name': u'1b386c986f9d06fd0a0dda70c3b8ade9.mpdf.tkn.txt', 'doc_id': u'1b386c986f9d06fd0a0dda70c3b8ade9.mpdf.tkn', 'file_id': u'1b386c986f9d06fd0a0dda70c3b8ade9.mpdf.tkn', 'event_nuggets': [{'span': (205, 228), 'event_id': 'E1', 'text': u'handed maximum sentence', 'span_confidence': 1.0, 'type_full': u'Justice_Sentence', 'tokens': [u'handed', u'maximum', u'sentence'], 'type_confidence': 1.0, 'realis_confidence': 1.0, 'char_offsets': [[205, 211], [212, 219], [220, 228]], 'realis': u'Actual'}, {'span': (241, 250), 'event_id': 'E2', 'text': u'sentences', 'span_confidence': 1.0, 'type_full': u'Justice_Sentence', 'tokens': [u'sentences'], 'type_confidence': 1.0, 'realis_confidence': 1.0, 'char_offsets': [[241, 250]], 'realis': u'Actual'}, {'span': (291, 297), 'event_id': 'E3', 'text': u'prison', 'span_confidence': 1.0, 'type_full': u'Justice_Arrest-Jail', 'tokens': [u'prison'], 'type_confidence': 1.0, 'realis_confidence': 1.0, 'char_offsets': [[291, 297]], 'realis': u'Generic'}, {'spa

#BeginOfDocument 04bfe2831596d665b1585d8bf7bedd47
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E32	219	pardon	Justice_Pardon	Generic
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E73	254	charged	Justice_Charge-Indict	Actual
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E187	238	executions	Justice_Execute	Generic
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E199	297	amnesties	Justice_Pardon	Actual
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E248	44	said	Contact_Broadcast	Actual
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E261	77	kidnapping	Conflict_Attack	Actual
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E289	194	reports	Contact_Broadcast	Actual
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E290	260	kidnapping	Conflict_Attack	Generic
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E308	325	disappearance	Conflict_Attack	Generic
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E325	53	charged	Justice_Charge-Indict	Actual
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E328	284	amnesty	Justice_Pardon	Generic
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E344	234	torture	Conflict_Attack	Generic
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E430	327	kidnapping	Movement_Transport-Person	Generic
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E460	164	taken	Movement_Transport-Person	Other
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E464	213	Amnesty	Justice_Pardon	Generic
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E598	204	held	Justice_Arrest-Jail	Actual
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E625	60	retired	Personnel_End-Position	Actual
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E642	327	kidnapping	Conflict_Attack	Generic
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E688	238	executions	Life_Die	Generic
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E774	260	kidnapping	Movement_Transport-Person	Generic
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E792	77	kidnapping	Movement_Transport-Person	Actual
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E839	201	tortured	Conflict_Attack	Actual
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E877	314	Amnesty	Justice_Pardon	Generic
brat_conversion	04bfe2831596d665b1585d8bf7bedd47	E999	142	detained	Justice_Arrest-Jail	Actual
@Coreference	C0	E73,E325
@Coreference	C1	E460,E792
@Coreference	C2	E598,E999
@Coreference	C3	E877,E32,E328,E464
@Coreference	C4	E642,E290
@Coreference	C5	E430,E774
#EndOfDocument

#BeginOfDocument <file_id>
<systen_name>	<file_id>	<event_key>	<span_begin>,<span_end>	<mention_text>	<event_type=Justice_Pardon>	<Realis=Generic>	<event_span_confidence>	<event_type_confidence>	<realis_confidence>
@Coreference	<coref_cluster_name=C1>	<event_key_1>,<event_key_2>
#EndOfDocument


In [55]:
def generate_output_file_conttent(docs_with_nuggets, system_name):
    output_str = ""
    for doc in docs_with_nuggets:
        doc_id = doc["doc_id"]
        output_str += "#BeginOfDocument\t%s\n" % doc_id
        
        for event_nugget in doc["event_nuggets"]:
            event_id = event_nugget["event_id"]
            mention_text = event_nugget["text"]
            span = event_nugget["span"]
            realis = event_nugget["realis"]
            type_full = event_nugget["type_full"]

            #Confidence score
            span_confidence = event_nugget["span_confidence"]
            realis_confidence = event_nugget["realis_confidence"]
            type_confidence = event_nugget["type_confidence"]
            
            output_str += "{system_name}\t{doc_id}\t{event_id}\t{span_begin},{span_end}\t{mention_text}\t{event_type}\t{realis}\t{event_span_confidence}\t{event_type_confidence}\t{realis_confidence}\n".format(
                                    system_name = system_name,
                                    doc_id = doc_id,
                                    event_id = event_id,
                                    span_begin = span[0],
                                    span_end = span[1],
                                    mention_text = mention_text,
                                    event_type = type_full,
                                    realis = realis,
                                    event_span_confidence = span_confidence,
                                    event_type_confidence = realis_confidence,
                                    realis_confidence = type_confidence)
            
        output_str += "#EndOfDocument\n"
        
        # Coreference
        if "coreference_list" in doc:
            for coref in doc["coreference_list"]:
                coref_id = coref["coref_id"]
                coref_events_keys = coref["events_keys"]
                coref_events_keys_str = string.join(coref_events_keys, ",")
                
                output_str += "@Coreference\t{coref_id}\t{events_keys}\n".format(coref_id = coref_id, 
                                        events_keys = coref_events_keys_str)
                
    
    return output_str 
        

In [56]:
str = generate_output_file_conttent([docs_with_nuggets[0]], system_name="aiphes_hd_t16") 
print str

#BeginOfDocument	1b386c986f9d06fd0a0dda70c3b8ade9.mpdf.tkn
aiphes_hd_t16	1b386c986f9d06fd0a0dda70c3b8ade9.mpdf.tkn	E1	205,228	handed maximum sentence	Justice_Sentence	Actual	1.0	1.0	1.0
aiphes_hd_t16	1b386c986f9d06fd0a0dda70c3b8ade9.mpdf.tkn	E2	241,250	sentences	Justice_Sentence	Actual	1.0	1.0	1.0
aiphes_hd_t16	1b386c986f9d06fd0a0dda70c3b8ade9.mpdf.tkn	E3	291,297	prison	Justice_Arrest-Jail	Generic	1.0	1.0	1.0
aiphes_hd_t16	1b386c986f9d06fd0a0dda70c3b8ade9.mpdf.tkn	E4	1274,1286	make a claim	Contact_Communicate	Actual	1.0	1.0	1.0
aiphes_hd_t16	1b386c986f9d06fd0a0dda70c3b8ade9.mpdf.tkn	E5	1303,1311	arrested	Justice_Arrest-Jail	Generic	1.0	1.0	1.0
aiphes_hd_t16	1b386c986f9d06fd0a0dda70c3b8ade9.mpdf.tkn	E6	1322,1332	imprisoned	Justice_Arrest-Jail	Generic	1.0	1.0	1.0
aiphes_hd_t16	1b386c986f9d06fd0a0dda70c3b8ade9.mpdf.tkn	E7	1354,1362	arrested	Justice_Arrest-Jail	Actual	1.0	1.0	1.0
aiphes_hd_t16	1b386c986f9d06fd0a0dda70c3b8ade9.mpdf.tkn	E8	1386,1396	extradited	Justice_Extradite	Actual	1.0	1.